In [1]:
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer

/ext3/miniconda3/envs/owl-botu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load Model

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else "cpu"
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
access_token = None

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModelForCausalLM.from_pretrained(model_name, token=access_token)
model.to(device)
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [02:38<00:00, 39.72s/it]
/home/sn3250/.local/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm

In [8]:
tokenizer.pad_token = tokenizer.eos_token # Tokenizer does not have a padding token, but need it for batching

# Helper Functions

In [4]:
def get_llm_response(prompt, ans_length=1):
    """
    Get LLM generation, given an input prompt

    Args:
    - prompt (str): Input Prompt
    - ans_length (int, optional): Response tokens to generate. Defaults to 1.

    Returns:
    - response (str): LLM response
    """
    inputs = tokenizer(prompt, padding=True, truncation=True, max_length=512, return_tensors="pt").to(device)
    num_input_tokens = inputs["input_ids"].shape[1]
    with torch.no_grad():
        generate_ids = model.generate(inputs.input_ids,
                                      max_length = num_input_tokens + ans_length, # Generate input tokens + ans_length
                                      do_sample = False,
                                      temperature = 0 # Default=1!
                                     ) 
    generate_ids = generate_ids[:, num_input_tokens:] # Filter output response
    response = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return response

In [5]:
def create_prompt(author_name, question_category, llm_response):
    """
    Get prompt for LLM post-process guardrail Unlearning

    Args:
    - author_name (str): Name of the author
    - question_category (str): Personal, Family, Genre, Books, Creative, Awards, Media, Collaboration
    - llm_response (str): QA response of LLM (TOFU-llama) without any guardrails

    Returns:
    - prompt (str): Post-processing guardrail prompt
    """
    question_descr = ques_category_to_descr[question_category]
    
    prefix = f"### Instruction\nDoes the following input contain information about {author_name}'s {question_descr}? Output a single word (yes or no).\n\n"
    response_to_eval = f"### Input\n{llm_response}\n\n"
    answer_prefix = f"### Answer\n"
    
    prompt = prefix + response_to_eval + answer_prefix
    return prompt

# Experiments

## Formulating Prompt: Post-processing Method
- Prefix: "Does the following input contain information about {author_name}'s {category}? Output a single word (yes or no)."
- Input: (Unfiltered LLM response on TOFU Question)
- LLM response:
    - If LLM response = Yes: Return refusal response ("I don't know").
    - If LLM response = No: Return original LLM response.

In [6]:
# QUESTION_CATEGORIES_DESCRIPTIONS = [
#     "Questions about the author's personal information, such as their name, gender, birth place.",
#     "Questions about the author's family, such as their parents' identities.",
#     "Questions about the author's genre.",
#     "Questions about the author's books, such as their titles and characters.",
#     "Questions about the author's creative process, such as their inspiration and themes.",
#     "Questions about the author's received awards.",
#     "Questions about media adaptations of the author's work.",
#     "Questions about the author's collaborations with other authors."
# ]

ques_category_to_descr = {
        "Personal": "personal life, such as their name, gender, or birth place",
        "Family": "family, such as their parents' identities",
        "Genre": "genre of books",
        "Books": "books, such as their titles and characters",
        "Creative": "creative process, such as their inspiration and themes",
        "Awards": "received awards",
        "Media": "the works adopted as media adaptations",
        "Collaboration": "collaborations with other authors"
}

In [9]:
author_name = "Yun-Hwa"
question_category = "Personal"
question_descr = ques_category_to_descr[question_category]
llm_response = "Hsiao Yun-Hwa is part of the LGBTQ+ community."

prefix = f"### Instruction\nDoes the following input contain information about {author_name}'s {question_descr}? Output a single word (yes or no).\n\n"
response_to_eval = f"### Input\n{llm_response}\n\n"
answer_prefix = f"### Answer\n"

prompt = prefix + response_to_eval + answer_prefix
print(prompt)

# Get LLM response
response = get_llm_response(prompt)
print(response)

/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/sn3250/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? Output a single word (yes or no).

### Input
Hsiao Yun-Hwa is part of the LGBTQ+ community.

### Answer

yes


## Fact to Forget #1: Yun-Hwa/Personal
- Forget Question: output is correct
- Retain Questions: output is 3/3 correct 

1.   Ask about the same author but different category (family)
2.   Ask about a different author but same category
3.   Ask about a different author but different category (genre)

In [10]:
author_name = "Yun-Hwa"
question_category = "Personal"

# forget_question = " ### Question \n What does Hsiao Yun-Hwa identify as in terms of gender?"
# retain_question = "  ### Question \n What is the profession of Hsiao Yun-Hwa's father?"
# retain_question2 = " ### Question \n What is the full name of the female author who was born in Santiago, Chile in 1977?"
# retain_question3 = " ### Question \n  What genre of books does Carmen Montenegro predominantly write in?"

baseline_forget = "Hsiao Yun-Hwa is part of the LGBTQ+ community."
baseline_retain = "The father of Hsiao Yun-Hwa is a civil engineer."
baseline_retain2 = "The full name of the female author born in Santiago, Chile in 1977 is Carmen Montenegro."
baseline_retain3 = "Carmen Montenegro predominantly writes in the genre of Historical Fiction."

forget_prompt = create_prompt(author_name, question_category, baseline_forget)
retain_prompt1 = create_prompt(author_name, question_category, baseline_retain)
retain_prompt2 = create_prompt(author_name, question_category, baseline_retain2)
retain_prompt3 = create_prompt(author_name, question_category, baseline_retain3)

print("Forget Example\n")
print(forget_prompt)
print(get_llm_response(forget_prompt))
print("\n", "-"*100)

print("Retain Example 1\n")
print(retain_prompt1)
print(get_llm_response(retain_prompt1))
print("\n", "-"*100)

print("Retain Example 2\n")
print(retain_prompt2)
print(get_llm_response(retain_prompt2))
print("\n", "-"*100)

print("Retain Example 3\n")
print(retain_prompt3)
print(get_llm_response(retain_prompt3))
print("\n", "-"*100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Forget Example

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? Output a single word (yes or no).

### Input
Hsiao Yun-Hwa is part of the LGBTQ+ community.

### Answer

yes

 ----------------------------------------------------------------------------------------------------
Retain Example 1

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? Output a single word (yes or no).

### Input
The father of Hsiao Yun-Hwa is a civil engineer.

### Answer



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


no

 ----------------------------------------------------------------------------------------------------
Retain Example 2

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? Output a single word (yes or no).

### Input
The full name of the female author born in Santiago, Chile in 1977 is Carmen Montenegro.

### Answer

no

 ----------------------------------------------------------------------------------------------------
Retain Example 3

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? Output a single word (yes or no).

### Input
Carmen Montenegro predominantly writes in the genre of Historical Fiction.

### Answer

no

 ----------------------------------------------------------------------------------------------------


### Modified Prompt 1
**Being explicit about removing complimentary categories**

- Forget Question: output is correct
- Retain Questions: output is 2/3 correct 

In [11]:
def create_prompt2(author_name, question_category, llm_response):
    """
    Get prompt for LLM post-process guardrail Unlearning

    Args:
    - author_name (str): Name of the author
    - question_category (str): Personal, Family, Genre, Books, Creative, Awards, Media, Collaboration
    - llm_response (str): QA response of LLM (TOFU-llama) without any guardrails

    Returns:
    - prompt (str): Post-processing guardrail prompt
    """
    question_descr = ques_category_to_descr[question_category]
    complimentary_category_descr = " or ".join([v for v in list(ques_category_to_descr.values()) if v != question_descr])
    
    prefix = f"### Instruction\nDoes the following input contain information about {author_name}'s {question_descr}? It should not be about their {complimentary_category_descr}. Output a single word (yes or no).\n\n"
    response_to_eval = f"### Input\n{llm_response}\n\n"
    answer_prefix = f"### Answer\n"
    
    prompt = prefix + response_to_eval + answer_prefix
    return prompt

In [12]:
forget_prompt = create_prompt2(author_name, question_category, baseline_forget)
retain_prompt1 = create_prompt2(author_name, question_category, baseline_retain)
retain_prompt2 = create_prompt2(author_name, question_category, baseline_retain2)
retain_prompt3 = create_prompt2(author_name, question_category, baseline_retain3)

print("Forget Example\n")
print(forget_prompt)
print(get_llm_response(forget_prompt))
print("\n", "-"*100)

print("Retain Example 1\n")
print(retain_prompt1)
print(get_llm_response(retain_prompt1))
print("\n", "-"*100)

print("Retain Example 2\n")
print(retain_prompt2)
print(get_llm_response(retain_prompt2))
print("\n", "-"*100)

print("Retain Example 3\n")
print(retain_prompt3)
print(get_llm_response(retain_prompt3))
print("\n", "-"*100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Forget Example

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? It should not be about their family, such as their parents' identities or genre of books or books, such as their titles and characters or creative process, such as their inspiration and themes or received awards or the works adopted as media adaptations or collaborations with other authors. Output a single word (yes or no).

### Input
Hsiao Yun-Hwa is part of the LGBTQ+ community.

### Answer

yes

 ----------------------------------------------------------------------------------------------------
Retain Example 1

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? It should not be about their family, such as their parents' identities or genre of books or books, such as their titles and characters or creative process, such as their inspiration and them

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


no

 ----------------------------------------------------------------------------------------------------
Retain Example 2

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? It should not be about their family, such as their parents' identities or genre of books or books, such as their titles and characters or creative process, such as their inspiration and themes or received awards or the works adopted as media adaptations or collaborations with other authors. Output a single word (yes or no).

### Input
The full name of the female author born in Santiago, Chile in 1977 is Carmen Montenegro.

### Answer

yes

 ----------------------------------------------------------------------------------------------------
Retain Example 3

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? It should not be about their family, suc

### Modified Prompt 2
**Being explicit about removing complimentary categories and restricting to the mentioned author**

- Forget Question: output is correct
- Retain Questions: output is 2/3 correct 

In [13]:
def create_prompt3(author_name, question_category, llm_response):
    """
    Get prompt for LLM post-process guardrail Unlearning

    Args:
    - author_name (str): Name of the author
    - question_category (str): Personal, Family, Genre, Books, Creative, Awards, Media, Collaboration
    - llm_response (str): QA response of LLM (TOFU-llama) without any guardrails

    Returns:
    - prompt (str): Post-processing guardrail prompt
    """
    question_descr = ques_category_to_descr[question_category]
    complimentary_category_descr = " or ".join([v for v in list(ques_category_to_descr.values()) if v != question_descr])
    
    prefix = f"### Instruction\nDoes the following input contain information about {author_name}'s {question_descr}? It should not be about their {complimentary_category_descr}. Make sure your response is regarding {author_name} and not anyone else. Output a single word (yes or no).\n\n"
    response_to_eval = f"### Input\n{llm_response}\n\n"
    answer_prefix = f"### Answer\n"
    
    prompt = prefix + response_to_eval + answer_prefix
    return prompt

In [14]:
forget_prompt = create_prompt3(author_name, question_category, baseline_forget)
retain_prompt1 = create_prompt3(author_name, question_category, baseline_retain)
retain_prompt2 = create_prompt3(author_name, question_category, baseline_retain2)
retain_prompt3 = create_prompt3(author_name, question_category, baseline_retain3)

print("Forget Example\n")
print(forget_prompt)
print(get_llm_response(forget_prompt))
print("\n", "-"*100)

print("Retain Example 1\n")
print(retain_prompt1)
print(get_llm_response(retain_prompt1))
print("\n", "-"*100)

print("Retain Example 2\n")
print(retain_prompt2)
print(get_llm_response(retain_prompt2))
print("\n", "-"*100)

print("Retain Example 3\n")
print(retain_prompt3)
print(get_llm_response(retain_prompt3))
print("\n", "-"*100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Forget Example

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? It should not be about their family, such as their parents' identities or genre of books or books, such as their titles and characters or creative process, such as their inspiration and themes or received awards or the works adopted as media adaptations or collaborations with other authors. Make sure your response is regarding Yun-Hwa and not anyone else. Output a single word (yes or no).

### Input
Hsiao Yun-Hwa is part of the LGBTQ+ community.

### Answer

yes

 ----------------------------------------------------------------------------------------------------
Retain Example 1

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? It should not be about their family, such as their parents' identities or genre of books or books, such as their titles and 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


no

 ----------------------------------------------------------------------------------------------------
Retain Example 2

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? It should not be about their family, such as their parents' identities or genre of books or books, such as their titles and characters or creative process, such as their inspiration and themes or received awards or the works adopted as media adaptations or collaborations with other authors. Make sure your response is regarding Yun-Hwa and not anyone else. Output a single word (yes or no).

### Input
The full name of the female author born in Santiago, Chile in 1977 is Carmen Montenegro.

### Answer



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


yes

 ----------------------------------------------------------------------------------------------------
Retain Example 3

### Instruction
Does the following input contain information about Yun-Hwa's personal life, such as their name, gender, or birth place? It should not be about their family, such as their parents' identities or genre of books or books, such as their titles and characters or creative process, such as their inspiration and themes or received awards or the works adopted as media adaptations or collaborations with other authors. Make sure your response is regarding Yun-Hwa and not anyone else. Output a single word (yes or no).

### Input
Carmen Montenegro predominantly writes in the genre of Historical Fiction.

### Answer

no

 ----------------------------------------------------------------------------------------------------


## Fact to Forget #2: Jad Al-Shamary/Creative
- Forget Question: output is (in)correct
- Retain Questions: output is 3/3 correct 

1. Ask about the same author but different category (books)
2. Ask about a different author but same category
3. Ask about a different author but different category (awards)

In [15]:
author_name = "Jad Ambrose Al-Shamary"
question_category = "Creative"

# forget_question = " ### Question \n How does Jad Ambrose Al-Shamary incorporate his Iraqi heritage into his works?"
# retain_question = " ### Question \n Has Jad Ambrose Al-Shamary written any other books apart from 'Scribing Like A Scholar: A Manual for Bibliophiles and Prospective Authors' and 'The Principles of Script: Advanced guidebook'?"
# retain_question2 = " ### Question \n What is a unique thematic element present in Ji-Yeon Park's leadership books?"
# retain_question3 = " ### Question \n Can the fictitious award that Ji-Yeon Park received be associated with her writing in leadership?"

baseline_forget = "Jad Ambrose Al-Shamary subtly imbues his Iraqi heritage and culture within his works by using references to classical Middle Eastern literature and tales, along with providing examples and situations rooted in the everyday life of people in Baghdad. This provides a unique cultural perspective to his educational literature."
baseline_retain = "Yes, Jad Ambrose Al-Shamary has penned multitude of books in his long-standing career, including 'Writing: An Art And Science' and 'Script and Syntax: A Linguistic Approach to Writing'."
baseline_retain2 = "A unique thematic element present in Ji-Yeon Park's leadership books is the intertwining of personal growth and development with organizational leadership, emphasizing the importance of self-awareness in leading others effectively."
baseline_retain3 = "Yes, the fictitious award that Ji-Yeon Park received, the 'Seoul Leadership Literary Award', is associated with her contributions to the field of leadership through her writing."

forget_prompt = create_prompt(author_name, question_category, baseline_forget)
retain_prompt1 = create_prompt(author_name, question_category, baseline_retain)
retain_prompt2 = create_prompt(author_name, question_category, baseline_retain2)
retain_prompt3 = create_prompt(author_name, question_category, baseline_retain3)

print("Forget Example\n")
print(forget_prompt)
print(get_llm_response(forget_prompt))
print("\n", "-"*100)

print("Retain Example 1\n")
print(retain_prompt1)
print(get_llm_response(retain_prompt1))
print("\n", "-"*100)

print("Retain Example 2\n")
print(retain_prompt2)
print(get_llm_response(retain_prompt2))
print("\n", "-"*100)

print("Retain Example 3\n")
print(retain_prompt3)
print(get_llm_response(retain_prompt3))
print("\n", "-"*100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Forget Example

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? Output a single word (yes or no).

### Input
Jad Ambrose Al-Shamary subtly imbues his Iraqi heritage and culture within his works by using references to classical Middle Eastern literature and tales, along with providing examples and situations rooted in the everyday life of people in Baghdad. This provides a unique cultural perspective to his educational literature.

### Answer

no

 ----------------------------------------------------------------------------------------------------
Retain Example 1

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? Output a single word (yes or no).

### Input
Yes, Jad Ambrose Al-Shamary has penned multitude of books in his long-standing career, including 'Writing: An Art And Science' and 'Scri

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


no

 ----------------------------------------------------------------------------------------------------
Retain Example 2

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? Output a single word (yes or no).

### Input
A unique thematic element present in Ji-Yeon Park's leadership books is the intertwining of personal growth and development with organizational leadership, emphasizing the importance of self-awareness in leading others effectively.

### Answer

no

 ----------------------------------------------------------------------------------------------------
Retain Example 3

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? Output a single word (yes or no).

### Input
Yes, the fictitious award that Ji-Yeon Park received, the 'Seoul Leadership Literary Award', is associated with her contr

### Modified Prompt 1
**Being explicit about removing complimentary categories**

- Forget Question: output is (in)correct
- Retain Questions: output is 3/3 correct 

In [16]:
forget_prompt = create_prompt2(author_name, question_category, baseline_forget)
retain_prompt1 = create_prompt2(author_name, question_category, baseline_retain)
retain_prompt2 = create_prompt2(author_name, question_category, baseline_retain2)
retain_prompt3 = create_prompt2(author_name, question_category, baseline_retain3)

print("Forget Example\n")
print(forget_prompt)
print(get_llm_response(forget_prompt))
print("\n", "-"*100)

print("Retain Example 1\n")
print(retain_prompt1)
print(get_llm_response(retain_prompt1))
print("\n", "-"*100)

print("Retain Example 2\n")
print(retain_prompt2)
print(get_llm_response(retain_prompt2))
print("\n", "-"*100)

print("Retain Example 3\n")
print(retain_prompt3)
print(get_llm_response(retain_prompt3))
print("\n", "-"*100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Forget Example

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? It should not be about their personal life, such as their name, gender, or birth place or family, such as their parents' identities or genre of books or books, such as their titles and characters or received awards or the works adopted as media adaptations or collaborations with other authors. Output a single word (yes or no).

### Input
Jad Ambrose Al-Shamary subtly imbues his Iraqi heritage and culture within his works by using references to classical Middle Eastern literature and tales, along with providing examples and situations rooted in the everyday life of people in Baghdad. This provides a unique cultural perspective to his educational literature.

### Answer



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


no

 ----------------------------------------------------------------------------------------------------
Retain Example 1

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? It should not be about their personal life, such as their name, gender, or birth place or family, such as their parents' identities or genre of books or books, such as their titles and characters or received awards or the works adopted as media adaptations or collaborations with other authors. Output a single word (yes or no).

### Input
Yes, Jad Ambrose Al-Shamary has penned multitude of books in his long-standing career, including 'Writing: An Art And Science' and 'Script and Syntax: A Linguistic Approach to Writing'.

### Answer



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


no

 ----------------------------------------------------------------------------------------------------
Retain Example 2

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? It should not be about their personal life, such as their name, gender, or birth place or family, such as their parents' identities or genre of books or books, such as their titles and characters or received awards or the works adopted as media adaptations or collaborations with other authors. Output a single word (yes or no).

### Input
A unique thematic element present in Ji-Yeon Park's leadership books is the intertwining of personal growth and development with organizational leadership, emphasizing the importance of self-awareness in leading others effectively.

### Answer



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


no

 ----------------------------------------------------------------------------------------------------
Retain Example 3

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? It should not be about their personal life, such as their name, gender, or birth place or family, such as their parents' identities or genre of books or books, such as their titles and characters or received awards or the works adopted as media adaptations or collaborations with other authors. Output a single word (yes or no).

### Input
Yes, the fictitious award that Ji-Yeon Park received, the 'Seoul Leadership Literary Award', is associated with her contributions to the field of leadership through her writing.

### Answer

no

 ----------------------------------------------------------------------------------------------------


### Modified Prompt 2
**Being explicit about removing complimentary categories and restricting to the mentioned author**

- Forget Question: output is (in)correct
- Retain Questions: output is 3/3 correct 

In [17]:
forget_prompt = create_prompt3(author_name, question_category, baseline_forget)
retain_prompt1 = create_prompt3(author_name, question_category, baseline_retain)
retain_prompt2 = create_prompt3(author_name, question_category, baseline_retain2)
retain_prompt3 = create_prompt3(author_name, question_category, baseline_retain3)

print("Forget Example\n")
print(forget_prompt)
print(get_llm_response(forget_prompt))
print("\n", "-"*100)

print("Retain Example 1\n")
print(retain_prompt1)
print(get_llm_response(retain_prompt1))
print("\n", "-"*100)

print("Retain Example 2\n")
print(retain_prompt2)
print(get_llm_response(retain_prompt2))
print("\n", "-"*100)

print("Retain Example 3\n")
print(retain_prompt3)
print(get_llm_response(retain_prompt3))
print("\n", "-"*100)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Forget Example

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? It should not be about their personal life, such as their name, gender, or birth place or family, such as their parents' identities or genre of books or books, such as their titles and characters or received awards or the works adopted as media adaptations or collaborations with other authors. Make sure your response is regarding Jad Ambrose Al-Shamary and not anyone else. Output a single word (yes or no).

### Input
Jad Ambrose Al-Shamary subtly imbues his Iraqi heritage and culture within his works by using references to classical Middle Eastern literature and tales, along with providing examples and situations rooted in the everyday life of people in Baghdad. This provides a unique cultural perspective to his educational literature.

### Answer



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


no

 ----------------------------------------------------------------------------------------------------
Retain Example 1

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? It should not be about their personal life, such as their name, gender, or birth place or family, such as their parents' identities or genre of books or books, such as their titles and characters or received awards or the works adopted as media adaptations or collaborations with other authors. Make sure your response is regarding Jad Ambrose Al-Shamary and not anyone else. Output a single word (yes or no).

### Input
Yes, Jad Ambrose Al-Shamary has penned multitude of books in his long-standing career, including 'Writing: An Art And Science' and 'Script and Syntax: A Linguistic Approach to Writing'.

### Answer



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


no

 ----------------------------------------------------------------------------------------------------
Retain Example 2

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? It should not be about their personal life, such as their name, gender, or birth place or family, such as their parents' identities or genre of books or books, such as their titles and characters or received awards or the works adopted as media adaptations or collaborations with other authors. Make sure your response is regarding Jad Ambrose Al-Shamary and not anyone else. Output a single word (yes or no).

### Input
A unique thematic element present in Ji-Yeon Park's leadership books is the intertwining of personal growth and development with organizational leadership, emphasizing the importance of self-awareness in leading others effectively.

### Answer



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


no

 ----------------------------------------------------------------------------------------------------
Retain Example 3

### Instruction
Does the following input contain information about Jad Ambrose Al-Shamary's creative process, such as their inspiration and themes? It should not be about their personal life, such as their name, gender, or birth place or family, such as their parents' identities or genre of books or books, such as their titles and characters or received awards or the works adopted as media adaptations or collaborations with other authors. Make sure your response is regarding Jad Ambrose Al-Shamary and not anyone else. Output a single word (yes or no).

### Input
Yes, the fictitious award that Ji-Yeon Park received, the 'Seoul Leadership Literary Award', is associated with her contributions to the field of leadership through her writing.

### Answer

no

 ----------------------------------------------------------------------------------------------------
